# Featuring Enginnering Using Scikit-Learn

### Do You Want to Build a Snowman?

Let's prep some data for a model to predict the height of snowman!

<img src = "./images/olaf.jpeg">

#### Load Packages

In [1]:
# data analysis stack
import numpy as np
import pandas as pd

# data visualization stack
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')

# machine-learning stack
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    KBinsDiscretizer,
    PolynomialFeatures
)

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# miscellaneous
import warnings
warnings.filterwarnings("ignore")

#### Load Data

In [2]:
data = {
    'temp' : [-3, 5, 0, 7, 3, -1, 1, None, -6, 3, 0, -1, None, -2],
    'lunch' : ['soup', 'sandwich', 'soup', 'burger', 'sandwich', 'soup', 'cereal', 'salad', 'sandwich', 'burger', 'soup', 'cereal', 'burger', 'soup'],
    'dinner' : ['pizza', 'pizza', 'noodles', None, 'fishsticks', 'pizza', None, 'fishsticks', 'noodles', 'pizza', None, 'pizza', 'fishsticks', 'pizza'],
    'precipitation' : ['yes', 'no', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'no'],
    'heigth_snowman_cm' : [100, 0, 75, 0, 20, 25, 0, 35, 170, 0, 85, 85, 45, 0]
}

df_train = pd.DataFrame(data=data)
df_train

,temp,lunch,dinner,precipitation,heigth_snowman_cm
0,-3.0,soup,pizza,yes,100
1,5.0,sandwich,pizza,no,0
2,0.0,soup,noodles,yes,75
3,7.0,burger,None,yes,0
4,3.0,sandwich,fishsticks,yes,20
5,-1.0,soup,pizza,yes,25
6,1.0,cereal,None,no,0
7,NaN,salad,fishsticks,yes,35
8,-6.0,sandwich,noodles,yes,170
9,3.0,burger,pizza,no,0


#### Exercise
Transform the data above using scikit-learn tool in a way that is suitable to be used for modeling
+ **Separate the DataFrame `df_train` into `X_train` and `y_train`** 
   +  Our target variable is `height_snowman_cm`
+ **Preprocess `X_train`**:
  + Identify which variables are **binary**, **categorical** and  **numeric**
  + Check which variables have **missing values**
    + **Impute missing value** as needed using appropriate strategy
  + Determine if categorical variables have **non-numeric values**
    + **Encode categorical variables** using techniques such as one-hot encoding
  + Determine if numeric variables are on different scale
    + **Scale numeric variables**
+ **Create `X_train_fe`**:
    + Once the preprocessing steps are completed, compile the transformed columns into a new DataFrame called `X_train_fe`. 


In [62]:
# split  - mind spelling mistake "heigth"
# Feature matrix 
X_train = df_train.drop("heigth_snowman_cm", axis=1)

# Target column
y_train = df_train["heigth_snowman_cm"]



In [34]:
X_train.isna().sum()

temp             2
lunch            0
dinner           3
precipitation    0
dtype: int64

In [55]:
# Instantiating a SimpleImputer object
temp_imputer_train = SimpleImputer(strategy='median').set_output(transform='pandas')
temp_imputer_train

,"missing_values missing_values: int, float, str, np.nan, None or pandas.NA, default=np.nanThe placeholder for the missing values. All occurrences of`missing_values` will be imputed. For pandas' dataframes withnullable integer dtypes with missing values, `missing_values`can be set to either `np.nan` or `pd.NA`.",nan
,"strategy strategy: str or Callable, default='mean'The imputation strategy.- If ""mean"", then replace missing values using the mean along each column. Can only be used with numeric data.- If ""median"", then replace missing values using the median along each column. Can only be used with numeric data.- If ""most_frequent"", then replace missing using the most frequent value along each column. Can be used with strings or numeric data. If there is more than one such value, only the smallest is returned.- If ""constant"", then replace missing values with fill_value. Can be used with strings or numeric data.- If an instance of Callable, then replace missing values using the scalar statistic returned by running the callable over a dense 1d array containing non-missing values of each column... versionadded:: 0.20 strategy=""constant"" for fixed value imputation... versionadded:: 1.5 strategy=callable for custom value imputation.",'median'
,"fill_value fill_value: str or numerical value, default=NoneWhen strategy == ""constant"", `fill_value` is used to replace alloccurrences of missing_values. For string or object data types,`fill_value` must be a string.If `None`, `fill_value` will be 0 when imputing numericaldata and ""missing_value"" for strings or object data types.",None
,"copy copy: bool, default=TrueIf True, a copy of X will be created. If False, imputation willbe done in-place whenever possible. Note that, in the following cases,a new copy will always be made, even if `copy=False`:- If `X` is not an array of floating values;- If `X` is encoded as a CSR matrix;- If `add_indicator=True`.",True
,"add_indicator add_indicator: bool, default=FalseIf True, a :class:`MissingIndicator` transform will stack onto outputof the imputer's transform. This allows a predictive estimatorto account for missingness despite imputation. If a feature has nomissing values at fit/train time, the feature won't appear onthe missing indicator even if there are missing values attransform/test time.",False
,"keep_empty_features keep_empty_features: bool, default=FalseIf True, features that consist exclusively of missing values when`fit` is called are returned in results when `transform` is called.The imputed value is always `0` except when `strategy=""constant""`in which case `fill_value` will be used instead... versionadded:: 1.2",False


In [56]:
# Fit the variable imputer on the flipper_length_mm' and 'bill_depth_mm' columns of the training data
temp_imputer_train.fit(X_train[['temp']])

,"missing_values missing_values: int, float, str, np.nan, None or pandas.NA, default=np.nanThe placeholder for the missing values. All occurrences of`missing_values` will be imputed. For pandas' dataframes withnullable integer dtypes with missing values, `missing_values`can be set to either `np.nan` or `pd.NA`.",nan
,"strategy strategy: str or Callable, default='mean'The imputation strategy.- If ""mean"", then replace missing values using the mean along each column. Can only be used with numeric data.- If ""median"", then replace missing values using the median along each column. Can only be used with numeric data.- If ""most_frequent"", then replace missing using the most frequent value along each column. Can be used with strings or numeric data. If there is more than one such value, only the smallest is returned.- If ""constant"", then replace missing values with fill_value. Can be used with strings or numeric data.- If an instance of Callable, then replace missing values using the scalar statistic returned by running the callable over a dense 1d array containing non-missing values of each column... versionadded:: 0.20 strategy=""constant"" for fixed value imputation... versionadded:: 1.5 strategy=callable for custom value imputation.",'median'
,"fill_value fill_value: str or numerical value, default=NoneWhen strategy == ""constant"", `fill_value` is used to replace alloccurrences of missing_values. For string or object data types,`fill_value` must be a string.If `None`, `fill_value` will be 0 when imputing numericaldata and ""missing_value"" for strings or object data types.",None
,"copy copy: bool, default=TrueIf True, a copy of X will be created. If False, imputation willbe done in-place whenever possible. Note that, in the following cases,a new copy will always be made, even if `copy=False`:- If `X` is not an array of floating values;- If `X` is encoded as a CSR matrix;- If `add_indicator=True`.",True
,"add_indicator add_indicator: bool, default=FalseIf True, a :class:`MissingIndicator` transform will stack onto outputof the imputer's transform. This allows a predictive estimatorto account for missingness despite imputation. If a feature has nomissing values at fit/train time, the feature won't appear onthe missing indicator even if there are missing values attransform/test time.",False
,"keep_empty_features keep_empty_features: bool, default=FalseIf True, features that consist exclusively of missing values when`fit` is called are returned in results when `transform` is called.The imputed value is always `0` except when `strategy=""constant""`in which case `fill_value` will be used instead... versionadded:: 1.2",False


In [32]:
temp_imputed_train = temp_imputer_train.transform(X_train[['temp']])
temp_imputed_train

,temp
0,-3.0
1,5.0
2,0.0
3,7.0
4,3.0
5,-1.0
6,1.0
7,0.0
8,-6.0
9,3.0


In [16]:
# replace None with np.nan
X_train['dinner'] = X_train['dinner'].fillna(np.nan)


In [17]:
# Instantiating a SimpleImputer object
dinner_imputer_train = SimpleImputer(strategy='most_frequent')

In [18]:
# fit variable on dinner column
dinner_imputer_train.fit(X_train[['dinner']])

,"missing_values missing_values: int, float, str, np.nan, None or pandas.NA, default=np.nanThe placeholder for the missing values. All occurrences of`missing_values` will be imputed. For pandas' dataframes withnullable integer dtypes with missing values, `missing_values`can be set to either `np.nan` or `pd.NA`.",nan
,"strategy strategy: str or Callable, default='mean'The imputation strategy.- If ""mean"", then replace missing values using the mean along each column. Can only be used with numeric data.- If ""median"", then replace missing values using the median along each column. Can only be used with numeric data.- If ""most_frequent"", then replace missing using the most frequent value along each column. Can be used with strings or numeric data. If there is more than one such value, only the smallest is returned.- If ""constant"", then replace missing values with fill_value. Can be used with strings or numeric data.- If an instance of Callable, then replace missing values using the scalar statistic returned by running the callable over a dense 1d array containing non-missing values of each column... versionadded:: 0.20 strategy=""constant"" for fixed value imputation... versionadded:: 1.5 strategy=callable for custom value imputation.",'most_frequent'
,"fill_value fill_value: str or numerical value, default=NoneWhen strategy == ""constant"", `fill_value` is used to replace alloccurrences of missing_values. For string or object data types,`fill_value` must be a string.If `None`, `fill_value` will be 0 when imputing numericaldata and ""missing_value"" for strings or object data types.",None
,"copy copy: bool, default=TrueIf True, a copy of X will be created. If False, imputation willbe done in-place whenever possible. Note that, in the following cases,a new copy will always be made, even if `copy=False`:- If `X` is not an array of floating values;- If `X` is encoded as a CSR matrix;- If `add_indicator=True`.",True
,"add_indicator add_indicator: bool, default=FalseIf True, a :class:`MissingIndicator` transform will stack onto outputof the imputer's transform. This allows a predictive estimatorto account for missingness despite imputation. If a feature has nomissing values at fit/train time, the feature won't appear onthe missing indicator even if there are missing values attransform/test time.",False
,"keep_empty_features keep_empty_features: bool, default=FalseIf True, features that consist exclusively of missing values when`fit` is called are returned in results when `transform` is called.The imputed value is always `0` except when `strategy=""constant""`in which case `fill_value` will be used instead... versionadded:: 1.2",False


In [19]:
dinner_imputer_train.statistics_

array(['pizza'], dtype=object)

In [33]:
# Applying the transformation to the 'sex' column of the training data using the pre-fitted imputer.
dinner_imputed_train = dinner_imputer_train.transform(X_train[['dinner']])
dinner_imputed_train

array([['pizza'],
       ['pizza'],
       ['noodles'],
       ['pizza'],
       ['fishsticks'],
       ['pizza'],
       ['pizza'],
       ['fishsticks'],
       ['noodles'],
       ['pizza'],
       ['pizza'],
       ['pizza'],
       ['fishsticks'],
       ['pizza']], dtype=object)

In [ ]:
# impute numerical: temp, heigth

# ohe: lunch, dinner, precipitation

# scale: temp and heigth_snowman_cm

In [36]:
# one hot encode for lunch
lunch_ohencoder = OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=True)

In [38]:
lunch_ohencoder.fit(X=X_train[['lunch']])

,"categories categories: 'auto' or a list of array-like, default='auto'Categories (unique values) per feature:- 'auto' : Determine categories automatically from the training data.- list : ``categories[i]`` holds the categories expected in the ith column. The passed categories should not mix strings and numeric values within a single feature, and should be sorted in case of numeric values.The used categories can be found in the ``categories_`` attribute... versionadded:: 0.20",'auto'
,"drop drop: {'first', 'if_binary'} or an array-like of shape (n_features,), default=NoneSpecifies a methodology to use to drop one of the categories perfeature. This is useful in situations where perfectly collinearfeatures cause problems, such as when feeding the resulting datainto an unregularized linear regression model.However, dropping one category breaks the symmetry of the originalrepresentation and can therefore induce a bias in downstream models,for instance for penalized linear classification or regression models.- None : retain all features (the default).- 'first' : drop the first category in each feature. If only one category is present, the feature will be dropped entirely.- 'if_binary' : drop the first category in each feature with two categories. Features with 1 or more than 2 categories are left intact.- array : ``drop[i]`` is the category in feature ``X[:, i]`` that should be dropped.When `max_categories` or `min_frequency` is configured to groupinfrequent categories, the dropping behavior is handled after thegrouping... versionadded:: 0.21 The parameter `drop` was added in 0.21... versionchanged:: 0.23 The option `drop='if_binary'` was added in 0.23... versionchanged:: 1.1 Support for dropping infrequent categories.",'first'
,"sparse_output sparse_output: bool, default=TrueWhen ``True``, it returns a :class:`scipy.sparse.csr_matrix`,i.e. a sparse matrix in ""Compressed Sparse Row"" (CSR) format... versionadded:: 1.2 `sparse` was renamed to `sparse_output`",True
,"dtype dtype: number type, default=np.float64Desired dtype of output.",<class 'numpy.float64'>
,"handle_unknown handle_unknown: {'error', 'ignore', 'infrequent_if_exist', 'warn'}, default='error'Specifies the way unknown categories are handled during :meth:`transform`.- 'error' : Raise an error if an unknown category is present during transform.- 'ignore' : When an unknown category is encountered during transform, the resulting one-hot encoded columns for this feature will be all zeros. In the inverse transform, an unknown category will be denoted as None.- 'infrequent_if_exist' : When an unknown category is encountered during transform, the resulting one-hot encoded columns for this feature will map to the infrequent category if it exists. The infrequent category will be mapped to the last position in the encoding. During inverse transform, an unknown category will be mapped to the category denoted `'infrequent'` if it exists. If the `'infrequent'` category does not exist, then :meth:`transform` and :meth:`inverse_transform` will handle an unknown category as with `handle_unknown='ignore'`. Infrequent categories exist based on `min_frequency` and `max_categories`. Read more in the :ref:`User Guide `.- 'warn' : When an unknown category is encountered during transform a warning is issued, and the encoding then proceeds as described for `handle_unknown=""infrequent_if_exist""`... versionchanged:: 1.1 `'infrequent_if_exist'` was added to automatically handle unknown categories and infrequent categories... versionadded:: 1.6 The option `""warn""` was added in 1.6.",'ignore'
,"min_frequency min_frequency: int or float, default=NoneSpecifies the minimum frequency below which a category will beconsidered infrequent.- If `int`, categories with a smaller cardinality will be considered infrequent.- If `float`, categories with a smaller cardinality than `min_frequency * n_samples` will be considered infrequent... versionadded:: 1.1 Read more in the :ref:`User Guide `.",None
,"max_ca

In [40]:
lunch_encoded_train_sparse = lunch_ohencoder.transform(X=X_train[['lunch']])
lunch_encoded_train_sparse

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 11 stored elements and shape (14, 4)>

In [41]:
# Convert a sparse matrix to a dense matrix
lunch_encoded_train_dense = lunch_encoded_train_sparse.todense()
lunch_encoded_train_dense

matrix([[0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 1.]])

In [42]:
# one hot encode for dinner
dinner_ohencoder = OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=True)

In [43]:
dinner_ohencoder.fit(X=X_train[['dinner']])

,"categories categories: 'auto' or a list of array-like, default='auto'Categories (unique values) per feature:- 'auto' : Determine categories automatically from the training data.- list : ``categories[i]`` holds the categories expected in the ith column. The passed categories should not mix strings and numeric values within a single feature, and should be sorted in case of numeric values.The used categories can be found in the ``categories_`` attribute... versionadded:: 0.20",'auto'
,"drop drop: {'first', 'if_binary'} or an array-like of shape (n_features,), default=NoneSpecifies a methodology to use to drop one of the categories perfeature. This is useful in situations where perfectly collinearfeatures cause problems, such as when feeding the resulting datainto an unregularized linear regression model.However, dropping one category breaks the symmetry of the originalrepresentation and can therefore induce a bias in downstream models,for instance for penalized linear classification or regression models.- None : retain all features (the default).- 'first' : drop the first category in each feature. If only one category is present, the feature will be dropped entirely.- 'if_binary' : drop the first category in each feature with two categories. Features with 1 or more than 2 categories are left intact.- array : ``drop[i]`` is the category in feature ``X[:, i]`` that should be dropped.When `max_categories` or `min_frequency` is configured to groupinfrequent categories, the dropping behavior is handled after thegrouping... versionadded:: 0.21 The parameter `drop` was added in 0.21... versionchanged:: 0.23 The option `drop='if_binary'` was added in 0.23... versionchanged:: 1.1 Support for dropping infrequent categories.",'first'
,"sparse_output sparse_output: bool, default=TrueWhen ``True``, it returns a :class:`scipy.sparse.csr_matrix`,i.e. a sparse matrix in ""Compressed Sparse Row"" (CSR) format... versionadded:: 1.2 `sparse` was renamed to `sparse_output`",True
,"dtype dtype: number type, default=np.float64Desired dtype of output.",<class 'numpy.float64'>
,"handle_unknown handle_unknown: {'error', 'ignore', 'infrequent_if_exist', 'warn'}, default='error'Specifies the way unknown categories are handled during :meth:`transform`.- 'error' : Raise an error if an unknown category is present during transform.- 'ignore' : When an unknown category is encountered during transform, the resulting one-hot encoded columns for this feature will be all zeros. In the inverse transform, an unknown category will be denoted as None.- 'infrequent_if_exist' : When an unknown category is encountered during transform, the resulting one-hot encoded columns for this feature will map to the infrequent category if it exists. The infrequent category will be mapped to the last position in the encoding. During inverse transform, an unknown category will be mapped to the category denoted `'infrequent'` if it exists. If the `'infrequent'` category does not exist, then :meth:`transform` and :meth:`inverse_transform` will handle an unknown category as with `handle_unknown='ignore'`. Infrequent categories exist based on `min_frequency` and `max_categories`. Read more in the :ref:`User Guide `.- 'warn' : When an unknown category is encountered during transform a warning is issued, and the encoding then proceeds as described for `handle_unknown=""infrequent_if_exist""`... versionchanged:: 1.1 `'infrequent_if_exist'` was added to automatically handle unknown categories and infrequent categories... versionadded:: 1.6 The option `""warn""` was added in 1.6.",'ignore'
,"min_frequency min_frequency: int or float, default=NoneSpecifies the minimum frequency below which a category will beconsidered infrequent.- If `int`, categories with a smaller cardinality will be considered infrequent.- If `float`, categories with a smaller cardinality than `min_frequency * n_samples` will be considered infrequent... versionadded:: 1.1 Read more in the :ref:`User Guide `.",None
,"max_ca

In [44]:
dinner_encoded_train_sparse = dinner_ohencoder.transform(X=X_train[['dinner']])
dinner_encoded_train_sparse

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 11 stored elements and shape (14, 3)>

In [45]:
# Convert a sparse matrix to a dense matrix
dinner_encoded_train_dense = dinner_encoded_train_sparse.todense()
dinner_encoded_train_dense

matrix([[0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 0.],
        [0., 1., 0.]])

In [48]:
# map precipitation to binary numeric values
X_train['precipitation'] = X_train['precipitation'].map({'yes': 1, 'no': 0})


In [51]:
# Ensure numeric columns are 2D
temp_array = np.array(temp_imputed_train)  # shape (n_samples, 1)

# Ensure encoded matrices are dense NumPy arrays
lunch_array = np.array(lunch_encoded_train_dense)
dinner_encoded_array = np.array(dinner_encoded_train_dense)

precipitation_array = np.array(X_train['precipitation']).reshape(-1, 1)



In [58]:
# Combine all preprocessed features
X_train_combined = np.hstack([
    temp_array,
    dinner_array,
    lunch_array,
    dinner_encoded_array,
    precipitation_array
])


In [54]:
numeric_cols = ['temp']
categorical_cols_imputed = ['dinner', 'lunch']
precipitation_col = ['precipitation']


In [42]:
lunch_ohe_cols = ['lunch_'+str(i) for i in range(lunch_array.shape[1])]
dinner_ohe_cols = ['dinner_'+str(i) for i in range(dinner_encoded_array.shape[1])]


In [55]:
all_columns = numeric_cols + categorical_cols_imputed + lunch_ohe_cols + dinner_ohe_cols + precipitation_col


In [59]:
import pandas as pd

X_train_fe = pd.DataFrame(X_train_combined, columns=all_columns, index=X_train.index)


ValueError: Shape of passed values is (14, 8), indices imply (14, 10)

In [60]:
X_train_fe

,temp,dinner,lunch_0,lunch_1,lunch_2,lunch_3,dinner_0,dinner_1
0,-3.0,pizza,0.0,0.0,0.0,1.0,0.0,1.0
1,5.0,pizza,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,noodles,0.0,0.0,0.0,1.0,1.0,0.0
3,7.0,pizza,0.0,0.0,0.0,0.0,0.0,1.0
4,3.0,fishsticks,0.0,0.0,1.0,0.0,0.0,0.0
5,-1.0,pizza,0.0,0.0,0.0,1.0,0.0,1.0
6,1.0,pizza,1.0,0.0,0.0,0.0,0.0,1.0
7,0.0,fishsticks,0.0,1.0,0.0,0.0,0.0,0.0
8,-6.0,noodles,0.0,0.0,1.0,0.0,1.0,0.0
9,3.0,pizza,0.0,0.0,0.0,0.0,0.0,1.0
